In [15]:
import os, time, json, requests, pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import constants as constants

In [27]:
# Getting release date 
game_titles = pd.read_parquet("../data/game_list.parquet")
API_KEY = constants.API_KEY
RELEASE_DATE_URL = "https://api.isthereanydeal.com/games/info/v2"
CACHE_DIR = "../data_raw/history"
OUT_CSV = "../data/game_info.csv"
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)

def retry_session():
    s = requests.Session()
    r = Retry(total=5, backoff_factor=0.6,
             status_forcelist=[429, 500, 502, 503, 504],
             allowed_methods=["GET", "POST"])
    s.mount("https://", HTTPAdapter(max_retries=r))
    return s

def fetch_game_data(uuid, session=None):
    session = session or retry_session()
    cache_path = os.path.join(CACHE_DIR, f"{uuid}.json")
    if os.path.exists(cache_path):
        with open(cache_path, "r", encoding="utf-8") as f:
            return json.load(f)

    params = { "key": API_KEY, "id": uuid }
    resp = session.get(RELEASE_DATE_URL, params=params, timeout=30)
    resp.raise_for_status()
    data = resp.json()

    with open(cache_path, "w", encoding="utf-8") as f:
        json.dump(data, f)
    return data

def append_rows(rows):
    df = pd.DataFrame(rows)
    if df.empty: return
    exists = os.path.exists(OUT_CSV)
    df.to_csv(OUT_CSV, mode="a", index=False, header=not exists)

def process_all(uuids, rpm=60):
    session = retry_session()
    rows_buffer = []
    window = 60.0 / max(rpm, 1)
    for i, uid in enumerate(uuids, 1):
        try:
            data = fetch_game_data(uid)
            steam_review = next(
                (r for r in data.get("reviews", []) if r.get("source") == "Steam"),
                {}
            )
            row = {
                "itad_uuid": uid,
                "type": data.get("type"),
                "achievements": data.get("achievements"),
                "mature": data.get("mature"),
                "release_date": data.get("releaseDate"),
                "rank": data.get("stats", {}).get("rank"),
                "collected": data.get("stats", {}).get("collected"),
                "steam_score": steam_review.get("score"),
                "steam_review_count": steam_review.get("count"),
                "tags": data.get("tags" or [])
            }
            rows_buffer.append(row)
        except requests.HTTPError as e:
            print(f"[{i}/{len(uuids)}] {uid} -> HTTP {e.response.status_code}")
        except Exception as e:
            print(f"[{i}/{len(uuids)}] {uid} -> ERROR {e}")
            
         # flush periodically to avoid losing progress
        if i % 50 == 0:
            append_rows(rows_buffer)
            rows_buffer.clear()
        time.sleep(window)

    # final flush
    append_rows(rows_buffer)


In [21]:
uuids = game_titles["itad_uuid"].dropna().unique().tolist()
print(len(uuids))

8523


In [22]:
process_all(uuids, rpm=60)

In [28]:
# Getting game history

API_KEY = constants.API_KEY
RELEASE_DATE_URL = "https://api.isthereanydeal.com/games/history/v2"
CACHE_DIR = "../data_raw/history"
OUT_CSV = "../data/game_history.csv"
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)

def retry_session():
    s = requests.Session()
    r = Retry(total=5, backoff_factor=0.6,
             status_forcelist=[429, 500, 502, 503, 504],
             allowed_methods=["GET", "POST"])
    s.mount("https://", HTTPAdapter(max_retries=r))
    return s

def fetch_game_history(uuid, shops=61, since="1998-01-01T00:00:00Z", session=None):
    session = session or retry_session()
    cache_path = os.path.join(CACHE_DIR, f"{uuid}.json")
    if os.path.exists(cache_path):
        with open(cache_path, "r", encoding="utf-8") as f:
            return json.load(f)

    params = { 
        "key": API_KEY,      
        "id": uuid,
        "shops": shops,
        "since": since,
     }
    resp = session.get(RELEASE_DATE_URL, params=params, timeout=30)
    resp.raise_for_status()
    data = resp.json()

    with open(cache_path, "w", encoding="utf-8") as f:
        json.dump(data, f)
    return data

def append_rows(rows):
    df = pd.DataFrame(rows)
    if df.empty: return
    exists = os.path.exists(OUT_CSV)
    df.to_csv(OUT_CSV, mode="a", index=False, header=not exists)

def process_all(uuids, rpm=60):
    session = retry_session()
    rows_buffer = []
    window = 60.0 / max(rpm, 1)
    for i, uid in enumerate(uuids, 1):
        try:
            data = fetch_game_history(uid, session=session)
            rows = []
            for entry in data:
                rows.append({
                    "itad_uuid": uid,
                    "timestamp": entry.get("timestamp"),
                    "shop_id": entry.get("shop", {}).get("id"),
                    "shop_name": entry.get("shop", {}).get("name"),
                    "price": entry.get("deal", {}).get("price", {}).get("amount"),
                    "regular": entry.get("deal", {}).get("regular", {}).get("amount"),
                    "currency": entry.get("deal", {}).get("price", {}).get("currency"),
                    "discount_pct": entry.get("deal", {}).get("cut"),
                })
            rows_buffer.extend(rows)
            
        except requests.HTTPError as e:
            print(f"[{i}/{len(uuids)}] {uid} -> HTTP {e.response.status_code}")
        except Exception as e:
            print(f"[{i}/{len(uuids)}] {uid} -> ERROR {e}")

        if i % 50 == 0 and rows_buffer:
            append_rows(rows_buffer)
            rows_buffer.clear()
        time.sleep(window)
        
    if rows_buffer:
        append_rows(rows_buffer)

In [29]:
process_all(uuids, rpm=60)